# Module 8: Working with a small signal

If you are collecting atomic and magnetic PDF simultaneously, the mPDF signal is often much smaller than the atomic PDF, sometimes even on the level of the noise in the atomic PDF fit. One way to deal with this is to collect data at a higher temperature where the mPDF signal is absent (e.g. well above the magnetic ordering temperature), do a regular atomic PDF fit to the high-temperature data, and subtract that fit residual from the fit residual you get when performing the atomic PDF fit to the low-temperature data, where the mPDF signal is small but nonzero. This allows you to subtract out temperature-independent (and therefore presumably nonmagnetic) noise or errors in your data, providing a cleaner mPDF signal to work with. We will work through an example of this approach in this module. 

The material we will use for our example is the multiferroic system Sr0.55Ba0.45MnO3, which undergoes an antiferromagnetic transition around 200 K. Mn is the only magnetic atom. We collected neutron PDF data on the NOMAD beamline at several temperature between 90 K and 500 K. We will do an mPDF fit to the 90 K data, using the data collected at 360 K as our high-temperature reference measurement for the temperature subtraction. The atomic PDF fits have already been done in PDFgui.

The following files will be relevant for this module:
 - SBMOfit_PDFgui_90K.fgr: best-fit atomic PDF at 90 K
 - SBMOfit_PDFgui_360K.fgr: best-fit atomic PDF at 360 K
 - struc_SBMO.stru: refined structure produced by PDFgui

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

from diffpy.mpdf import *
from diffpy.structure import loadStructure

%matplotlib notebook 
# run this cell twice

### Inspect the structure and determine the magnetic propagation/basis vectors

Load in the refined structure file using loadStructure(). Print out the structure to determine which index corresponds to the magnetic atom. Convince yourself that the Mn atoms sit on the vertices of a simple pseudocubic lattice (pseudocubic because there is a very slight tetragonal distortion).

In [ ]:
struc = loadStructure('files/struc_SBMO.stru')
print(struc)

The magnetic order is known to be G-type, which means each Mn spin is antiferromagnetically coupled with its 6 nearest neighbors. Further more, the spins are known to be oriented up and down along the c axis. Use this information to determine the propagation vector and basis vector to describe this magnetic structure.

### Calculate and plot the expected mPDF pattern so we know what to look for

In [ ]:
### Make the MagSpecies and MagStructure

svec = ???? # basis vector
kvec = ???? # propagation vector
mspec = MagSpecies(struc=struc,strucIdxs=[????], basisvecs=svec,kvecs=np.array([0.5,0.5,0.5]),rmaxAtoms=25,
                   ffparamkey='Mn4',origin=struc[2].xyz_cartn)

# Very important: Note that we passed a value for the "origin" attribute when creating our MagSpecies.
# This specifies the origin relative to which the propagation vector modulates the spins. By default,
# it is the origin of the direct lattice, but if the magnetic atom is not located precisely at the
# origin, then this will result in non-physical complex spins. Therefore, we set the origin equal
# to the Cartesian coordinates of the Mn atom in the unit cell.

mstruc = MagStructure()
mstruc.loadSpecies(mspec)
mstruc.makeAll() ### populates the spin and atom arrays

### Make the MPDFcalculator

mc = MPDFcalculator(magstruc=mstruc,qdamp=0.025)

### Calculate and plot the mPDF
rmag, gmagNrm, gmag = mc.calc(both=True)
mc.plot(both=True)

### Inspect the fit residuals and look for evidence for a magnetic PDF signal

In [ ]:
### Read in the fits done at 90 K and 360 K
r, gobs, gcalc, gdiff = read_fgr('files/SBMOfit_PDFgui_90K.fgr')
r360, gobs360, gcalc360, gdiff360 = read_fgr('files/SBMOfit_PDFgui_360K.fgr')

### Plot the 90 K fit residual with the calculated mPDF (scaled to match the data)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('r ($\AA$)')
ax.set_ylabel('G ($\AA^{-2}$)')
ax.plot(r, gdiff)
ax.plot(rmag, gmag*????) # play around with this until you find a good scaling factor
plt.tight_layout()
plt.show()

It looks like there could be some rough agreement between the fit residual and the calculated mPDF. (Note that the high-frequency wiggles in the fit residual cannot possibly be magnetic in origin, since the mPDF is guaranteed to be much broader than that.)

Let's try to clean up the signal by subtracting the high-temperature data.

In [ ]:
gdiffSub = gdiff - gdiff360

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('r ($\AA$)')
ax.set_ylabel('G ($\AA^{-2}$)')
ax.plot(r, gdiffSub)
ax.plot(rmag, gmag*????)
plt.tight_layout()
plt.show()

This is starting to look promising! We can do one more trick to improve the subtraction: run the optimizedSubtraction() routine, which attempts to account for thermal expansion and thermal broadening by applying stretching and broadening operations to the low-temperature data. Execute the cell below to learn more about how it works.

In [ ]:
optimizedSubtraction?

In [ ]:
r_opt, gdiff_opt = optimizedSubtraction(????)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('r ($\AA$)')
ax.set_ylabel('G ($\AA^{-2}$)')
ax.plot(r_opt, gdiff_opt)
ax.plot(rmag, gmag*????)
plt.tight_layout()
plt.show()

We can also apply an aesthetic correction. Since we know the high-frequency noise cannot be magnetic in origin, we can filter it out using the smoothData() function. Learn more about it below:

In [ ]:
smoothData?

We will just use the standard step-function cutoff. Let's try 6 A^-1 and see how that looks.

In [ ]:
gsmooth = smoothData(r_opt, gdiff_opt, 6)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('r ($\AA$)')
ax.set_ylabel('G ($\AA^{-2}$)')
ax.plot(r_opt, gsmooth)
ax.plot(rmag, gmag*????)
plt.tight_layout()
plt.show()

Not bad! Now we can do an actual fit to improve the agreement. Note that we should fit to the original data, not the smoothed data, but we can always do the visual comparison with the smoothed data to see how it looks.

For fitting parameters, let's choose the two scale factors (paraScale and ordScale) and the two spherical angles to allow us to refine the spin direction. To keep things simple, we'll just use least_squares directly without implementing the full diffpy.srfit framework.

In [ ]:
### Update rmin and rmax to agree with the data
mc.rmin = r_opt.min()
mc.rmax = r_opt.max()

### Define masks for the up and down spins

upSpins = ????
downSpins = ~upSpins

# Define the mPDF function that will be evaluated in the fit
def mpdf(p):
    ordscale, parascale, th, phi = p
    mc.ordScale = ordscale
    mc.paraScale = parascale
    # components of our new spin vector
    sx = ????
    sy = ????
    sz = ????
    newSvec = np.array([sx, sy, sz])
    mstruc.spins[upSpins] = 1.0*newSvec
    mstruc.spins[downSpins] = -1.0*newSvec
    return mc.calc(both=True)[2] # just returning the unnormalized mPDF


def residual(p, data):
    return data - mpdf(p)

# set starting parameter values (same order as defined in the mpdf(p) function)
p0 = [0.01, 0.01, np.pi/2, 0] 

# set reasonable lower and upper bounds for the parameters
lb = [0, 0, 0, -np.pi]
ub = [10, 10, np.pi, np.pi]

# run the optimization
opt = least_squares(residual, p0, bounds=[lb,ub], args=(gdiff_opt,)) # passing gdiff_opt as the "data" argument in residual()

# print the refined parameter values
print(opt.x)

# calculate the best-fit mPDF to plot later
magcalc = mpdf(opt.x)

# print out the refined spin direction
print(mstruc.spins[0])

Now we plot the results. We'll include both the atomic and magnetic PDF fits.

In [ ]:
baseline = 1.2 * gobs.min()
overall_residual = gdiff_opt - magcalc

fig = plt.figure(figsize=(6,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
ax.plot(r, gobs, 'bo', label="G(r) data",markerfacecolor='none', markeredgecolor='b')
ax.plot(r, gcalc+magcalc, 'r-', lw=1.5, label="G(r) fit")
ax.plot(r, gdiff_opt + baseline,marker='o',mec='Gray',mfc='None',linestyle='None')
ax.plot(r, magcalc + baseline, marker='None',color='Blue',lw=2)
ax.plot(r, overall_residual+1.4*baseline, 'g-')
ax.plot(r, np.zeros_like(r) + 1.4*baseline, 'k:')


ax.set_xlabel(r"r ($\mathdefault{\AA}$)")
ax.set_ylabel(r"G ($\mathdefault{\AA^{-2}}$)")

ax2.plot(r, gdiff_opt, linestyle='-', lw=0.5, color='k') # unsmoothed version
ax2.plot(r, gsmooth, marker='o',mec='Gray',mfc='None',linestyle='None', label='G$_{\mathdefault{mag}}$') #smoothed version
ax2.plot(r, magcalc, 'b-', lw=1.5, label="G$_{\mathdefault{mag}}$ fit")
ax.legend()
ax2.legend()
ax2.set_xlabel(r"r ($\mathdefault{\AA}$)")
ax2.set_ylabel(r"G$_{\mathdefault{mag}}$ ($\mathdefault{\AA^{-2}}$)")

plt.show()

In [ ]:
plt.close('all')

Let's check the ordered moment that corresponds to our fit. This is a good sanity check; if we get an unreasonably large or small value, it could mean that there is something wrong with the fit.

In [ ]:
nucScale = 0.613

calculate_ordered_moment(mc, nucScale)

This seems like a reasonable value for Mn4+. Another successful mPDF fit!